In [1]:
import numpy as np
import pandas as pd
import argparse
from sklearn.tree import DecisionTreeClassifier, plot_tree, _tree
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
import pydotplus
from matplotlib import pyplot as plt
from labels import mac_to_label
import json
import math
import copy

In [2]:
inputfile = "data.csv"
outputfile = "tree.json"

# Training set X and Y
Set1 = pd.read_csv(inputfile)
Set = Set1.values.tolist()
X = [i[0:3] for i in Set]
Y =[i[3] for i in Set]

# Test set Xt and Yt
Set2 = pd.read_csv(inputfile)
Sett = Set2.values.tolist()
Xt = [i[0:3] for i in Set]
Yt =[i[3] for i in Set]

# prepare training and testing set
X = np.array(X)
Y = np.array(Y)
Xt = np.array(Xt)
Yt = np.array(Yt)

print(f"dataset size: {len(X)}")

# decision tree fit
dt = DecisionTreeClassifier(max_depth = 5)
dt.fit(X, Y)
Predict_Y = dt.predict(X)
print(f"pre-nudge train accuracy: {accuracy_score(Y, Predict_Y)}")

Predict_Yt = dt.predict(Xt)
print(f"pre-nudge test accuracy: {accuracy_score(Yt, Predict_Yt)}")

dataset size: 1894144
pre-nudge train accuracy: 0.879490682862549
pre-nudge test accuracy: 0.879490682862549


In [3]:
def nudge_threshold_power_2(threshold: float):
    threshold = math.floor(threshold)
    bit_len = threshold.bit_length()
    if threshold <= 2:
        nudged_value = threshold
    elif threshold & (1 << (bit_len - 2)):
        nudged_value = 1 << (bit_len)
    else:
        nudged_value = 1 << (bit_len - 1)
    return nudged_value

In [4]:
def nudge_threshold_n_significant_bits(threshold: float, n_sig_bits: int):
    threshold = math.floor(threshold)
    
    bit_len = threshold.bit_length()
    if bit_len - 1 <= n_sig_bits:
        nudged_value = threshold
    else:
        mask = ((1 << n_sig_bits) - 1) << (threshold.bit_length() - n_sig_bits)
        nudged_value = threshold & mask
        if threshold & (1 << (bit_len - 1 - n_sig_bits)):
            nudged_value += (1 << (bit_len - n_sig_bits))
            
    return nudged_value

In [5]:
def nudge_threshold_max_n_bits(threshold: float, n_bits: int):
    threshold = math.floor(threshold)
    if n_bits == 0:
        return threshold
    
    mask = pow(2, 32) - 1 ^ ((1 << n_bits) - 1)
    nudged_value = threshold & mask
    if threshold & (1 << (n_bits - 1)):
        nudged_value += (1 << (n_bits))
            
    return nudged_value

In [6]:
def crawl_tree(tree: _tree, node):
    # postfix traversal on non-leaf nodes
    flag = 0
    if tree.children_left[node] != -1:
        crawl_tree(tree, tree.children_left[node])
        flag |= 1
    if tree.children_right[node] != -1:
        crawl_tree(tree, tree.children_right[node])
        flag |= 1

    if flag:
        # can choose `nudge_threshold_power_2`, `nudge_threshold_n_significant_bits`, or `nudge_threshold_max_n_bits`
        tree.threshold[node] = nudge_threshold_max_n_bits(tree.threshold[node], 2)

original_tree = dt.tree_
nudged_tree = copy.deepcopy(original_tree)
crawl_tree(nudged_tree, 0)

In [7]:
dt.tree_ = nudged_tree
Predict_Y = dt.predict(X)
print(f"post-nudge train accuracy: {accuracy_score(Y, Predict_Y)}")

Predict_Yt = dt.predict(Xt)
print(f"post-nudge test accuracy: {accuracy_score(Yt, Predict_Yt)}")

post-nudge train accuracy: 0.8767189822949047
post-nudge test accuracy: 0.8767189822949047


In [8]:
# output the tree
def get_lineage(tree, feature_names):
    data = {"features": {}, "paths": [], "classes": set()}

    thresholds = tree.tree_.threshold
    features = [feature_names[i] for i in tree.tree_.feature]
    left = tree.tree_.children_left
    right = tree.tree_.children_right
    value = tree.tree_.value

    data["classes"] = list(tree.classes_)
 
    # get ids of child nodes
    idx = np.argwhere(left == -1)[:, 0]
    # traverse the tree and get the node information
    def recurse(left, right, child, lineage=None):
        if lineage is None:
            lineage = [child]
        if child in left:
            parent = np.where(left == child)[0].item()
            split = 'l'
        else:
            parent = np.where(right == child)[0].item()
            split = 'r'
        
        lineage.append((parent, split, thresholds[parent], features[parent]))
        if parent == 0:
            lineage.reverse()
            return lineage
        else:
            return recurse(left, right, parent, lineage)

    id = 0
    for j, child in enumerate(idx):
        clause = []
        for node in recurse(left, right, child):
                if len(str(node)) < 3:
                    continue
                direction = node[1]
                threshold = node[2]
                feature = node[3]
                if direction == "l": # feature <= threshold
                    clause.append({"feature": feature, "operation": "<=", "value": threshold})
                else: # direction == "r" # feature > threshold
                    threshold
                    clause.append({"feature": feature, "operation": ">", "value": threshold})
                    
        a = list(value[node][0])
        ind = a.index(max(a))
        clause = {"conditions": clause, "classification": ind, "id": id}
        data["paths"].append(clause)
        id += 1

    for i, fe in enumerate(features):
        if tree.tree_.feature[i] != _tree.TREE_UNDEFINED:
            if not fe in data["features"]:
                data["features"][fe] = []
            data["features"][fe].append(thresholds[i])

    return data

In [9]:
# get feature names
feature_names = Set1.columns
file = open(outputfile, "w+")
lineage = get_lineage(dt, feature_names)

class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)

file.write(json.dumps(lineage, indent = 4, cls=SetEncoder))
file.close()

In [10]:
fig = plt.figure(figsize=(100,100))
_ = plot_tree(dt, filled=True)